In [1]:
# %% === < Global Setting: Time and Seed > ===
import time
import numpy as np
local_time = time.strftime("%Y%m%d_%H%M%S", time.localtime(time.time()))
print("Local Time (UTC+8):", local_time)
seed = int(round(1000000*np.random.random()))
print("Seed:", seed)
np.random.seed(seed)

Local Time (UTC+8): 20210609_123715
Seed: 52991


In [2]:
# %% === < Windowed Image I/O > ===
import os
import numpy as np
from pydicom import dcmread
import cv2

data_dst_train = 'Data/TrainData_HemBonSub/'
data_dst_valid = 'Data/ValidData_HemBonSub/'
data_src = 'Data/TrainingData/'

category_dir = os.listdir(data_src)

for category in category_dir:
    print(category)
    
    output_cat_path_train = os.path.join(data_dst_train,category)
    if not os.path.exists(output_cat_path_train):
        os.makedirs(output_cat_path_train)
    output_cat_path_valid = os.path.join(data_dst_valid,category)
    if not os.path.exists(output_cat_path_valid):
        os.makedirs(output_cat_path_valid)
    
    input_cat_path = os.path.join(data_src,category)
    input_dir = os.listdir(input_cat_path)
    
    num_file = 0
    for file in input_dir:
        input_path = os.path.join(input_cat_path,file)
        
        # === dicom file reading
        ds = dcmread(input_path)
        dcm = ds.pixel_array.copy()
        dcm[dcm < ds.RescaleSlope] = ds.RescaleSlope
        dcm = ds.RescaleIntercept + dcm*ds.RescaleSlope
        dcm = dcm.astype(np.int16)
        len_row = ds.Rows
        len_col = ds.Columns
        
        # === hemorrhage window
        mask_hemorrhage = np.ones([len_row,len_col])
        mask_hemorrhage[dcm < 65] = 0
        mask_hemorrhage[dcm > 100] = 0
        value_hemorrhage = (dcm*mask_hemorrhage)
        
        # === bone window
        mask_bone = np.ones([len_row,len_col])
        mask_bone[dcm < 300] = 0
        value_bone = (dcm*mask_bone)
        
        # === brain window
        mask_brain = np.ones([len_row,len_col])
        mask_brain[dcm < 0] = 0
        mask_brain[dcm > 80] = 0
        value_brain = (dcm*mask_brain)
        
        # === subdural window
        mask_subdural = np.ones([len_row,len_col])
        mask_subdural[dcm < -15] = 0
        mask_subdural[dcm > 115] = 0
        value_subdural = (dcm*mask_subdural)
        
        # === RGB image writing
        R = value_hemorrhage
        G = value_bone
        B = value_subdural
        img = cv2.merge([B,G,R]).astype(np.uint8)
#         img = cv2.merge([B,G,2*R]).astype(np.uint8)
        
        if num_file < 700:
            output_path = os.path.join(output_cat_path_train,file)
            cv2.imwrite(output_path[0:-4]+'.png', img)
        else:
            output_path = os.path.join(output_cat_path_valid,file)
            cv2.imwrite(output_path[0:-4]+'.png', img)
        
        num_file = num_file + 1
print('Finish!!!')

epidural
healthy
intraparenchymal
intraventricular
subarachnoid
subdural
Finish!!!


In [3]:
# %% === < Windowed Image I/O > ===
import os
import numpy as np
from pydicom import dcmread
import cv2

data_dst = 'Data/TestData_HemBonSub/'
data_src = 'Data/TestingData/'

category_dir = os.listdir(data_src)

for category in category_dir:
    print(category)
    
    output_cat_path = os.path.join(data_dst,category)
    if not os.path.exists(output_cat_path):
        os.makedirs(output_cat_path)
    
    input_cat_path = os.path.join(data_src,category)
    input_dir = os.listdir(input_cat_path)
    
    for file in input_dir:
        input_path = os.path.join(input_cat_path,file)
        
        # === dicom file reading
        ds = dcmread(input_path)
        dcm = ds.pixel_array.copy()
        dcm[dcm < ds.RescaleSlope] = ds.RescaleSlope
        dcm = ds.RescaleIntercept + dcm*ds.RescaleSlope
        dcm = dcm.astype(np.int16)
        len_row = ds.Rows
        len_col = ds.Columns
        
        # === hemorrhage window
        mask_hemorrhage = np.ones([len_row,len_col])
        mask_hemorrhage[dcm < 65] = 0
        mask_hemorrhage[dcm > 100] = 0
        value_hemorrhage = (dcm*mask_hemorrhage)
        
        # === bone window
        mask_bone = np.ones([len_row,len_col])
        mask_bone[dcm < 300] = 0
        value_bone = (dcm*mask_bone)
        
        # === brain window
        mask_brain = np.ones([len_row,len_col])
        mask_brain[dcm < 0] = 0
        mask_brain[dcm > 80] = 0
        value_brain = (dcm*mask_brain)
        
        # === subdural window
        mask_subdural = np.ones([len_row,len_col])
        mask_subdural[dcm < -15] = 0
        mask_subdural[dcm > 115] = 0
        value_subdural = (dcm*mask_subdural)
        
        # === RGB image writing
        R = value_hemorrhage
        G = value_bone
        B = value_subdural
        img = cv2.merge([B,G,R]).astype(np.uint8)
#         img = cv2.merge([B,G,2*R]).astype(np.uint8)
        
        output_path = os.path.join(output_cat_path,file)
        cv2.imwrite(output_path[0:-4]+'.png', img)

print('Finish!!!')

epidural
healthy
intraparenchymal
intraventricular
subarachnoid
subdural
Finish!!!
